## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)



In [2]:
#  Import and read the charity_data.csv.
worksheet = gc.open('charity_data').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

In [3]:
# read in dataframe, reset first row to header
df = pd.DataFrame.from_records(rows)
df.columns = df.iloc[0]
df = df[1:]
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
1,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
2,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
3,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
4,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
5,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df2 = df.drop(['EIN', 'NAME'], 1)
df2.head()

<ipython-input-4-c0072f51f8d4>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df2 = df.drop(['EIN', 'NAME'], 1)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
1,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
2,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
3,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
4,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
5,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Determine the number of unique values in each column.
df2.nunique()

0
APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = df2['APPLICATION_TYPE'].value_counts()

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
df2
application_types_to_replace = list(application_counts[application_counts < 300].index)

# Replace in dataframe
for app in application_types_to_replace:
    df2['APPLICATION_TYPE'] = df2['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df2['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
classification_counts = df2['CLASSIFICATION'].value_counts()

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
df2['CLASSIFICATION'].value_counts().loc[lambda x : x>1]


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 250].index)


# Replace in dataframe
for cls in classifications_to_replace:
    df2['CLASSIFICATION'] = df2['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df2['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1197
C7000      777
C1700      287
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummy_df = pd.get_dummies(df2)
application_dummy_df = application_dummy_df.drop(['IS_SUCCESSFUL_1'], 1)
application_dummy_df.head()


<ipython-input-11-b8ba0198d718>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  application_dummy_df = application_dummy_df.drop(['IS_SUCCESSFUL_1'], 1)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ASK_AMT_997073,ASK_AMT_9971,ASK_AMT_99821,ASK_AMT_99859,ASK_AMT_99877,ASK_AMT_99879,ASK_AMT_9990,ASK_AMT_999468,ASK_AMT_99982,IS_SUCCESSFUL_0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Split our preprocessed data into our features and target arrays

y = application_dummy_df['IS_SUCCESSFUL_0'].values
X = application_dummy_df.drop(["IS_SUCCESSFUL_0"], axis='columns').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=8792))

# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=30, activation="sigmoid"))

# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                703440    
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 705,901
Trainable params: 705,901
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 10s 11ms/step - loss: 0.6610 - accuracy: 0.6685
Epoch 2/100
804/804 [==============================] - 9s 11ms/step - loss: 0.4672 - accuracy: 0.7984
Epoch 3/100
804/804 [==============================] - 8s 10ms/step - loss: 0.4266 - accuracy: 0.8099
Epoch 4/100
804/804 [==============================] - 9s 11ms/step - loss: 0.4363 - accuracy: 0.8019
Epoch 5/100
804/804 [==============================] - 7s 9ms/step - loss: 0.4242 - accuracy: 0.8097
Epoch 6/100
804/804 [==============================] - 9s 11ms/step - loss: 0.4163 - accuracy: 0.8119
Epoch 7/100
804/804 [==============================] - 9s 11ms/step - loss: 0.4252 - accuracy: 0.8027
Epoch 8/100
804/804 [==============================] - 8s 10ms/step - loss: 0.4225 - accuracy: 0.8057
Epoch 9/100
804/804 [==============================] - 11s 13ms/step - loss: 0.4169 - accuracy: 0.8125
Epoch 10/100
804/804 [==============================] - 7s 9ms/step - loss: 0.410

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.8162 - accuracy: 0.7251 - 1s/epoch - 5ms/step
Loss: 0.8162209391593933, Accuracy: 0.7251312136650085


In [19]:
# Export our model to HDF5 file
nn_model.to_hdf('AlphabetSoupCharity.h5')

AttributeError: ignored